# Pacman in Automation

**Authors**: 
* Corey Craddock - 
* Braden Hogan -
* Justin Rentie - _jurentie@rams.colostate.edu_
* Wesley Turner - 

**Date Submitted**: 12/12/2017

**Description**: Create a game of Pacman with controls to vary levels of Pacman and Ghost intelligence and mazes.

# Introduction

For our CS 440 Final Project, we chose to implement artificial intelligence algorithms that we learned in class throughout this semester to run the classic arcade game Pacman in full automation. We chose this as our final project for multiple reasons.

First, all four of us have deeply enjoyed learning artificial intelligence using games this semester, and we wanted our last project to reflect this. 

Second, we have already seen the AI algorithms, which are Iterative Deepening Search and Q Table Reinforcement Learning, implemented within a game setting. We expected that implementing these algorithms for a new game to be both an obtainable goal, as well as a challenging one.

Last, in our initial research for this project, we found that implementing artificial intelligence with Pacman was a common project for computer scientists, taking the form of a rite of a passage for anyone deeply interested in artificial intelligence.

Over the course of the last few weeks, we have worked diligently to implement a game of Pacman with these algorithms. Below, we discuss our implementation for the framework of the project, the object used to save the state and players, the Iterative Deepening Search algorithm with Pacman and Ghosts, and the Reinforcement Learning algorithm with Pacman.

# Table of Contents
* **[Framework - Corey Craddock](#Framework---Corey-Craddock)**
* **[Gameboard - Braden Hogan](#Gameboard---Braden-Hogan)
* [Iterative Deepening Search - Justin Rentie](#Iterative-Deepening-Search---Justin-Rentie)
* [Reinforcement Learning - Wesley Turner](#Reinforcement-Learning---Wesley-Turner)
* [Methods](#Methods)
* [Results - Corey Craddock](#Results---Corey-Craddock)
* [References](#References)

# Implementation

We've outlined thoughts on design and decisions that we made in implementing each of these sections here.

## Framework - Corey Craddock

Pacman and ghosts are both represented by classes we created. Using examples from past lectures, each of these objects has the following base methods:

takeAction(board, action) - Returns the state if action is taken on board

actions(board) - Returns all possible actions (up, down, left, right)

They also contain useful information such as lives, current location, and respawn points. 

Another useful class created is PlayGame.py. This is what we use to run a game. Its definition is below, with examples used farther down in the notebook

In [21]:
def startGame(board, p, ghostsAvailable, Q, intelligenceLevel=3, pacmanIntelligent=False, verbose=True):
    # Current Score of the game stored as an Integer
    score = 0
    turn = 1
    dead = False
    ghosts = []

    while not p.gameOver(board):
        beginGhosts = copy.deepcopy(ghostsAvailable)
        if dead:
            if verbose:
                print("You died!")
            dead = False
        if verbose:
            print("Lives:", p.getLives(), "\tDots left:", board.dotsLeft, "\tLocation:", p.location, "\tTurn:", turn, "\tScore:", score)
            print(board, end='')
            print("\nActions available:", p.actions(board))
        turn, ghosts, p, board, score, dead = runSingleTurn(turn, ghosts, beginGhosts, intelligenceLevel, p, board, score, dead, "", pacmanIntelligent, Q)
    # Game over
    if verbose:
        if p.lives == 0:
            print("Game over: You died!")
        else:
            print("Game over: You ate all the dots with", p.getLives(), "lives left!")
    return (turn, score, p.getLives(), board.dotsLeft)

Lastly, the end results are very important. In order to speed up testing a wide variety of new board states, we created a seperate class, Stats.py. Stats has a method for every board used, and simply calling the method from main, will run a game on that board. In order to get a wide variety of states tested, we decided to collect results for the following:

 - 3 different boards
 - Each board tested with 1, 2 and 3 ghosts
 - Each (board, ghost) combo tested with ghost intelligence levels of 1, 2 and 3 
         - 1: ghosts use shortest distance to hunt Pacman
         - 2 and 3: ghosts hunt for Pacman using IDS depths 4 and 8, respectively
 - Each (board, ghost, intelligence) combo tested with both IDS and Q Table for Pacman's AI
 
 The results are at the bottom of the page

## Gameboard - Braden Hogan

#### Introduction

The design and implementation of gameboard was to approach the problem in an object oriented modular way. The result was a representation of state that could be implemented with any underlying data structure.

#### Resources

We used the Python documentation to understand and make use of the operator overloading provided within the python language.

#### Implementation

Within the implementation of BoardGame the important methods are: `move`, `__getitem__`, `__str__`. These functions are designed to control the interactions of other portions of the code with the state representation.


##### `move` implementation:

In [ ]:
    def move(self, target, x, y):
        tarX = target.location[0]
        tarY = target.location[1]
        if isinstance(target, P.PacMan):
            if self.board[x][y] is '.':
             self.dotsLeft = self.dotsLeft - 1
             self.removeDot(x, y)
             self.board[x][y] = 'p'
             self.board[tarX][tarY] = ' '
             target.location = x,y
            else:
             self.board[x][y] = 'p'
             self.board[tarX][tarY] = ' '
             target.location = x,y
        else:
            if self.board[x][y] is '.':
             self.board[x][y] = 'g'
             if target.onDot:
                 self.board[tarX][tarY] = '.'
                 target.onDot = False
             else: self.board[tarX][tarY] = ' '
             target.onDot = True
             target.location = x,y
            else:
             self.board[x][y] = 'g'
             if target.onDot:
                 self.board[tarX][tarY] = '.'
                 target.onDot = False
             else: self.board[tarX][tarY] = ' '
             target.location = x,y

With this implementation of the state the first major problem was how other entities would interact with the state. The first interaction was one of the entities needing to move. Within the implementation there were two cases since there were two major entities within the program, pacman and ghost. The only changes made to the state occur through this method which is used within `takeaction` for both ghost and pacman. The details of implementation for each entity is as follows:

- Pacman: If the object was a pacman then there were two cases if there was a dot or not. If there was a dot then the number of dots left was decremented and pacman was moved and had the location updated. Otherwise pacman moved and left a blank space behind.


- Ghost: This implementation was slightly more complicated since ghosts do not consume dots. Thus the only adjustment from the pacman move was an addition of a Boolean to tell the ghost whether or not to replace its previous position with a dot.





##### `__getitem__` implementation:

In [ ]:
    def __getitem__(self, item):
        x = item[0]
        y = item[1]
        return self.board[x][y]

Access was the second major problem for objects when interacting with the state. The decision was made to implement the assessor operator to act on a tuple of the (x,y) pair wanting to be accessed. Within the logic of the program it was decided as well that since lists were used for the base logic that (0,0) would represent the top left of the current maze. This method controlled the access of the underlying state representation by returning the value at the given (x,y) tuple.

##### `__str__` implementation:

In [2]:
    def __str__(self):
        value = ''
        for i in range(len(self.board)):
             for j in range(len(self.board[i])):
                value += self.board[i][j]
             value += '\n'
        return value

The next important portion of the implementation was printing the board's current state in a clean and efficient way. The decision was made to use the to string method for the `GameBoard` object. This was used since it allowed the state to be printed easily using python's print method.

##### Summary:

Overall the goal was a completed implementation where the state was modularized enough that the underlying representation could be changed without the remainder of the program needing to be updated or modified. Using the modularity created the state could be represented by a numpy array or as a dictionary with changes to only the GameBoard class.


## Iterative Deepening Search - Justin Rentie

##### Overview:
It was our intention in designing our Pacman project that we would implement two different kinds of Artificial Intelligence for PacMan. We wanted to implement Iterative-Deepening Search for the first.

##### Resources:
We referenced a couple lecture notes from class in implementing IDS for Pacman. We used Lecture 05 Iterative Deepening and Other Uninformed Search Methods and 06 Python Implementation of Iterative Deepening as an example of how to implement basic IDS for searching through a tree of possible states. We also relied on examples from Assignment 02.  

_Links to these references are found [below](#References)._

##### Implementation:
We used Iterative Deepening Search to search within a given depth limit from possible actions that can be taken from pacman at any given state. If pacman locates a dot within the given depth limit the path to that dot is returned. 

The only major change that needed to be implemented was in the case of approaching a dot near any ghosts. In this situation we added a function called `fearFactor()` which will return `run` from `depthLimitedSearch()` which then in turn calls a function which removes possible moves from Pacman that could lead him towards a ghost. This results in Pacman effectively "running away" from the ghosts. 

This will go on until Pacman finds a dot within the given depth limit that is not in the direction of a ghost.

In [4]:
    # PacMan "blood hunter" AI. Copied from Ghost.
    # Helps Intelligent Move in finding the best direction to take to get to nearest dot
    def depthLimitedSearch(self, board, ghosts, closestDots, actions, takeAction, depthLimit):
        if PacMan.fearFactor(self, ghosts):
            return 'run'

        for dot in closestDots:
            if self.location == dot.location:
                return []

        if depthLimit == 0:
            return "cutoff"

        cutOffOccurred = False
        for action in PacMan.actions(self, board):
            copyBoard = copy.deepcopy(board)
            copySelf = copy.deepcopy(self)
            takeAction(copySelf, copyBoard, action)
            result = PacMan.depthLimitedSearch(copySelf, copyBoard, ghosts, closestDots, actions, takeAction,
                                              depthLimit - 1)

            if result is "cutoff":
                cutOffOccurred = True
            elif result is not "failure":
                return action
        if cutOffOccurred:
            return "cutoff"
        else:
            return "failure"

    # Causes the ghost to scan through the board, making the most intelligent shortest path decision
    def intelligentMove(self, board, ghosts, maxDepth=4):
        closestDots = PacMan.getClosestDots(self, board)

        for dot in closestDots:
            if self.location == dot.location:
                return

        for depth in range(maxDepth):
            result = PacMan.depthLimitedSearch(self, board, ghosts, closestDots, PacMan.actions, PacMan.takeAction,
                                              depth)
            if result != "cutoff" and result != "failure" and result != 'run':
                print("PacMan found intelligent move. Returning intelligentMove")
                return PacMan.takeAction(self, board, result)

        if(result == 'run'):
            PacMan.runFromGhost(self, board, ghosts,  PacMan.actions)
        else:
            PacMan.makeRandomMove(self, board)

### Example:

Example run of Pacman using Iterative-Deepening Search

In [19]:
import PlayGame as game
import PacMan as P
import Ghost as G
import Dot as d
import GameBoard as Board
import copy as copy

In [20]:
 # Really basic state to start with
state = [['=', '=', '=', '=', '=', '=', '=', '=', '=', '=', '='],
        ['|', ' ', ' ', ' ', ' ', 'G', ' ', ' ', ' ', ' ', '|'],
        ['|', ' ', '=', '=', '=', ' ', '=', '=', '=', ' ', '|'],
        ['|', ' ', '|', ' ', '|', ' ', '|', ' ', '|', ' ', '|'],
        ['|', ' ', '=', '=', '=', ' ', '=', '=', '=', ' ', '|'],
        ['|', '.', '.', '.', '.', '.', '.', '.', '.', 'P', '|'],
        ['=', '=', '=', '=', '=', '=', '=', '=', '=', '=', '=']]

In [21]:
# Start game with above state and 2 ghosts
board = Board.GameBoard(state)
# Create a Pacman object using this board
p = P.PacMan(board.pacManSpawnPt)

ghostsAvailable = [G.Ghost(board.ghostSpawnPt)]
intelligenceLevel = 3

# Initialize Q to empty array
Q = []

# Runs startGame with Q table and printing
game.startGame(board, p, ghostsAvailable, Q, intelligenceLevel, True, True)

Lives: 3 	Dots left: 8 	Location: (5, 9) 	Turn: 1 	Score: 0
|    G    |
| === === |
| | | | | |
| === === |
|........P|

Actions available: ['up', 'left']
PacMan found intelligent move. Returning intelligentMove
Lives: 3 	Dots left: 7 	Location: (5, 8) 	Turn: 2 	Score: 10
|    G    |
| === === |
| | | | | |
| === === |
|.......p |

Actions available: ['left', 'right']
PacMan found intelligent move. Returning intelligentMove
Lives: 3 	Dots left: 6 	Location: (5, 7) 	Turn: 3 	Score: 19
|    G    |
| === === |
| | | | | |
| === === |
|......p  |

Actions available: ['left', 'right']
PacMan found intelligent move. Returning intelligentMove
Lives: 3 	Dots left: 5 	Location: (5, 6) 	Turn: 4 	Score: 28
|         |
| ===g=== |
| | | | | |
| === === |
|.....p   |

Actions available: ['left', 'right']
PacMan found intelligent move. Returning intelligentMove
Lives: 3 	Dots left: 4 	Location: (5, 5) 	Turn: 5 	Score: 37
|         |
| === === |
| | |g| | |
| === === |
|....p    |

Actions available:

## Reinforcement Learning - Wesley Turner

##### Overview:
From the beginning, we knew we wanted to write a function to train a Q table for the states that Pacman would most likely move through in a game. We planned for this to be our most optimized form of a smart Pacman, as this intelligence made the most sense for our game.

##### Resources:
We utilized two different sources to implement this. The first was from a previous assignment in this course, Assignment 5. A link for this assignment page can be found below. This assignment had students implement training a Q table and testing it with the Towers of Hanoi puzzle. Although this was a good start, that puzzle is a one-person game, so our reinforcement would need to differ vastly. However, the overall structure and design for the trainQ function initially came from this assignment.

Secondly, we took inspiration for the reinforcement for Pacman from the lecture notes titled “Reinforcement Learning for Two-Player Games”. We knew we needed to augment the reinforcement for different actions that Pacman may take, and the implementation in this lecture was sufficient to start our brainstorming.

_Links to these references are found [below](#References)._

##### Implementation:
Despite these sources aiding us with design, we were still running into issues with what actions needed reinforcement. These were the actions we knew we could reinforce:

 - Change to score
 - Pacman dies
 - Pacman wins or loses
 - Pacman eats a dot

We tried a variety of combinations of these reinforcements. For the most part, these were blind tests: which combination of reinforcements would lead Pacman to have the best and fastest results? We ended up utilizing the implementation below:

In [3]:
    def trainQ(self, board, nRepetitions, learningRate, epsilonDecayFactor, ghostsAvailable, intelligenceLevel, verbose=False):
        maxGames = nRepetitions
        rho = learningRate
        epsilonDecayRate = epsilonDecayFactor
        epsilon = 1.0
        Q = {}
        scores = []

        for nGames in range(maxGames):
            if verbose:
                print("Game:", nGames, "; Starting game.")
            epsilon *= epsilonDecayRate
            step = 0
            state = copy.deepcopy(board)
            copySelf = copy.deepcopy(self)
            ghosts = []
            score = 0
            done = False

            while not done and not copySelf.gameOver(state):
                dead = False
                copyGhosts = copy.deepcopy(ghostsAvailable)
                step += 1
                copyState = copy.deepcopy(state)
                move = PacMan.epsilonGreedy(copySelf, epsilon, Q, copyState)

                _, ghosts, copySelf, stateNew, score, dead = PlayGame.runSingleTurn(step, ghosts, copyGhosts, intelligenceLevel, copySelf, copyState, score, dead, move)
                # Full return: turn, ghosts, p, board, score, dead

                #Initial value
                if PacMan.boardMoveTuple(copySelf, state, move) not in Q:
                    Q[PacMan.boardMoveTuple(copySelf, state, move)] = 0  # initial Q value for new state,move

                if stateNew.dotsLeft < state.dotsLeft:
                    #Pacman ate a dot. Medium positive reinforcement
                    Q[PacMan.boardMoveTuple(copySelf, state, move)] += 3
                else:
                    #Pacman did not eat a dot. Small negative reinforcement
                    Q[PacMan.boardMoveTuple(copySelf, state, move)] += -1
                if dead:
                    #Pacman lost a life. Large negative reinforcement
                    Q[PacMan.boardMoveTuple(copySelf, state, move)] = -10

                if step > 1:
                    Q[PacMan.boardMoveTuple(copySelf, stateOld, moveOld)] += rho * (Q[PacMan.boardMoveTuple(copySelf, state, move)] - Q[PacMan.boardMoveTuple(copySelf, stateOld, moveOld)])

                stateOld, moveOld, scoreOld = state, move, score
                state = stateNew
            if verbose:
                if copySelf.lives > 0:
                    print("Pacman Won!")
                else:
                    print("Pacman Lost!")
            scores.append(score)
        return Q, scores

This combination of reinforcements were tested to have the best results for Pacman, as it allows him to have some flexibility when choosing a state/move that is not necessarily taking the closest path to a dot (i.e. he needs to run away). We will be displaying the performance that we found in using trainQ and the Q table it creates in a section below.

### Example:

Example run of Pacman using training Q table

In [23]:
import PlayGame as game
import PacMan as P
import Ghost as G
import Dot as d
import GameBoard as Board
import copy as copy

In [24]:
 # Really basic state to start with
state = [['=', '=', '=', '=', '=', '=', '=', '=', '=', '=', '='],
        ['|', ' ', ' ', ' ', ' ', 'G', ' ', ' ', ' ', ' ', '|'],
        ['|', ' ', '=', '=', '=', ' ', '=', '=', '=', ' ', '|'],
        ['|', ' ', '|', ' ', '|', ' ', '|', ' ', '|', ' ', '|'],
        ['|', ' ', '=', '=', '=', ' ', '=', '=', '=', ' ', '|'],
        ['|', '.', '.', '.', '.', '.', '.', '.', '.', 'P', '|'],
        ['=', '=', '=', '=', '=', '=', '=', '=', '=', '=', '=']]

In [25]:
# Start game with above state and 2 ghosts
board = Board.GameBoard(state)
# Create a Pacman object using this board
p = P.PacMan(board.pacManSpawnPt)

ghostsAvailable = [G.Ghost(board.ghostSpawnPt)]
intelligenceLevel = 3

# Train Q for p
Q = []
# Trains Q table and prints each game
Q, scores = p.trainQ(board, 30, 0.5, 0.7, ghostsAvailable, intelligenceLevel, True)
print(scores)

# Runs startGame with Q table and printing
game.startGame(board, p, ghostsAvailable, Q, intelligenceLevel, False, True)

Game: 0 ; Starting game.
Pacman Lost!
Game: 1 ; Starting game.
Pacman Lost!
Game: 2 ; Starting game.
Pacman Lost!
Game: 3 ; Starting game.
Pacman Lost!
Game: 4 ; Starting game.
Pacman Lost!
Game: 5 ; Starting game.
Pacman Lost!
Game: 6 ; Starting game.
Pacman Lost!
Game: 7 ; Starting game.
Pacman Lost!
Game: 8 ; Starting game.
Pacman Won!
Game: 9 ; Starting game.
Pacman Won!
Game: 10 ; Starting game.
Pacman Won!
Game: 11 ; Starting game.
Pacman Won!
Game: 12 ; Starting game.
Pacman Won!
Game: 13 ; Starting game.
Pacman Won!
Game: 14 ; Starting game.
Pacman Won!
Game: 15 ; Starting game.
Pacman Won!
Game: 16 ; Starting game.
Pacman Won!
Game: 17 ; Starting game.
Pacman Won!
Game: 18 ; Starting game.
Pacman Won!
Game: 19 ; Starting game.
Pacman Won!
Game: 20 ; Starting game.
Pacman Won!
Game: 21 ; Starting game.
Pacman Won!
Game: 22 ; Starting game.
Pacman Won!
Game: 23 ; Starting game.
Pacman Won!
Game: 24 ; Starting game.
Pacman Won!
Game: 25 ; Starting game.
Pacman Won!
Game: 26 ; Sta

# Methods

Below are all methods and files in our project. We've decided to show who authored which section to show that we all worked as evenly as possible in implementing this project.
 
 - **Dot.py** - All methods authored by Justin Rentie
 - **GameBoard.py** - All methods authored by Braden Hogan
 - **Ghost.py**
     - init - Corey Craddock & Justin Rentie
     - actions - Corey Craddock & Braden Hogan
     - takeAction - Corey Craddock, Braden Hogan, & Justin Rentie
     - randomMove - Corey Craddock
     - takeActionShortestDistance - Wesley Turner
     - depthLimitedSearch - Wesley Turner
     - intelligentMove - Wesley Turner
 - **Pacman.py**
     - init - Corey Craddock & Justin Rentie
     - spawnPt - Corey Craddock
     - actions - Corey Craddock & Braden Hogan
     - takeAction - Corey Craddock, Braden Hogan, & Justin Rentie
     - calculateDistance - Justin Rentie
     - getClosestDot - Justin Rentie
     - directionToObj - Justin Rentie
     - fearFactor - Justin Rentie
     - depthLimitedSearch - Justin Rentie
     - intelligentMove - Justin Rentie
     - runFromGhost - Justin Rentie
     - makeRandomMove - Justin Rentie
     - gameOver - Corey Craddock
     - getLives - Corey Craddock
     - boardMoveTuple - Wesley Turner
     - useReinforcementTable - Wesley Turner
     - epsilonGreedy - Wesley Turner
     - trainQ - Wesley Turner
 - **PlayGame.py**
     - getDots - Justin Rentie
     - runSingleTurn - Wesley Turner
     - startGame - Corey Craddock & Wesley Turner
     - main - Corey Craddock & Wesley Turner
 - **Stats.py** - All meothods authored by Corey Craddock

# Results - Corey Craddock

In order to test our Pacman AI algorithms on a wide variety of states, we created a seperate class **Stats.py** to create multiple boards and run both the IDS and Q Table AI algorithms on each board, with a variety of ghosts and ghost intelligence levels. Here are our results. ** Warning: Running any of the code cells below will take 10-30 minutes to complete. **


** Board 1 **

First, we will start with a simple state, with a clear path. The state looks like this: ![this](board1.png "Board 1")

The Ghost on the board is the ghost spawn point, and the Pacman is the Pacman spawn point. We used an ASCII representation of the board as our visual during the project bulding.

Given this board, lets see how Pacman does with varying number of ghosts and varying ghost intelligence. We will try both of Pacman's AI algorithms on this board with 1-3 ghosts, each with a 1-3 intelligence level.

In [1]:
# Format: [<number of ghosts>, <ghost intelligence>] <pacMan AI>  <turns>  <score>  <lives left>  <moves explored>  <dots left>
import Stats as stats
print("Board 1:")
for i in range(1, 4):
    for j in range(1, 4):
        if i is 1 and j is 1:
            stats.board1(i, j, True, True)
        else:
            stats.board1(i, j, True, False)
        stats.board1(i, j, False, False)

Board 1:
|    G    |
| === === |
| | | | | |
| === === |
|........P|

	 [1, 1] IDS 30-average results:		Turns: 17 	Score: 64 	Lives left: 3 	Dots left: 0
	 [1, 1] Q Table Results:		Turns: 9 	Score: 73 	Lives left: 3 	Dots left: 0
	 [1, 2] IDS 30-average results:		Turns: 21 	Score: 62 	Lives left: 3 	Dots left: 0
	 [1, 2] Q Table Results:		Turns: 9 	Score: 73 	Lives left: 3 	Dots left: 0
	 [1, 3] IDS 30-average results:		Turns: 18 	Score: 63 	Lives left: 3 	Dots left: 0
	 [1, 3] Q Table Results:		Turns: 9 	Score: 73 	Lives left: 3 	Dots left: 0
	 [2, 1] IDS 30-average results:		Turns: 21 	Score: 43 	Lives left: 1 	Dots left: 1
	 [2, 1] Q Table Results:		Turns: 9 	Score: 73 	Lives left: 3 	Dots left: 0
	 [2, 2] IDS 30-average results:		Turns: 22 	Score: 41 	Lives left: 1 	Dots left: 1
	 [2, 2] Q Table Results:		Turns: 9 	Score: 73 	Lives left: 3 	Dots left: 0
	 [2, 3] IDS 30-average results:		Turns: 19 	Score: 62 	Lives left: 3 	Dots left: 0
	 [2, 3] Q Table Results:		Turns: 9 	Score: 73

Here are some graphs to help visualize the data above: ![this](board1Graphs.png "Board 1 Graphs")

The above code ran the board 9 times. Each turn it switched off between IDS (30 game average) and Q Table (trained 30 times) and increased the number of ghosts and their intelligence. From these results, we can see that with this board, Q Table performs better in every area. It always eats all the dots in 9 turns without losing any lives. IDS doesn't perform quite as well, but still mostly completes the board. The ghost intelligence doesn't seem to affect IDS very much, but the  number of ghosts have a big impact. As you add ghosts, the IDS algorithm takes more turns to complete, loses more lives, and has a lower score


** Board 2 **

Now we will try a new board, and see how that affects Pacman's performance. We quickly learned that increasing the boards size  drastically increases the time the Q Table completes. One board that we tried, we let the algorithms run for around ~6 hours overnight and they did not complete. Because of this restriction, the board size and structure will stay the same, but we will add new dots in different positions with future boards. I'm also limiting the number of ghosts on the third board to only 2 ghosts. The second board looks like this:![this](board2.png "Board 2")

I had to break up the testing into several cells here, as running them in the same cell takes a very long time to complete

In [10]:
# Format: [<number of ghosts>, <ghost intelligence>] <pacMan AI>  <turns>  <score>  <lives left>  <moves explored>  <dots left>
import Stats as stats
print("Board 2:")
stats.board2(1, 1, True, True)
stats.board2(1, 1, False, False)
stats.board2(1, 2, True, False)
stats.board2(1, 2, False, False)
print('', end='')

Board 2:
|.   G   .|
|.=== ===.|
|.| | | |.|
|.=== ===.|
|........P|

	 [1, 1] IDS 30-average results:		Turns: 32 	Score: 129 	Lives left: 3 	Dots left: 0
	 [1, 1] Q Table Results:		Turns: 21 	Score: 141 	Lives left: 3 	Dots left: 0
	 [1, 2] IDS 30-average results:		Turns: 31 	Score: 130 	Lives left: 3 	Dots left: 0
	 [1, 2] Q Table Results:		Turns: 48 	Score: 0 	Lives left: 0 	Dots left: 10


In [13]:
stats.board2(1, 3, True, False)
stats.board2(1, 3, False, False)
stats.board2(2, 1, True, False)
stats.board2(2, 1, False, False)
print('', end='')

	 [1, 3] IDS 30-average results:		Turns: 25 	Score: 136 	Lives left: 3 	Dots left: 0
	 [1, 3] Q Table Results:		Turns: 33 	Score: 29 	Lives left: 2 	Dots left: 0
	 [2, 1] IDS 30-average results:		Turns: 32 	Score: 43 	Lives left: 1 	Dots left: 6
	 [2, 1] Q Table Results:		Turns: 19 	Score: 0 	Lives left: 0 	Dots left: 8


In [14]:
stats.board2(2, 2, True, False)
stats.board2(2, 2, False, False)
stats.board2(2, 3, True, False)
stats.board2(2, 3, False, False)
print('', end='')

	 [2, 2] IDS 30-average results:		Turns: 32 	Score: 60 	Lives left: 1 	Dots left: 4
	 [2, 2] Q Table Results:		Turns: 49 	Score: 0 	Lives left: 0 	Dots left: 11
	 [2, 3] IDS 30-average results:		Turns: 28 	Score: 133 	Lives left: 3 	Dots left: 0
	 [2, 3] Q Table Results:		Turns: 25 	Score: 37 	Lives left: 2 	Dots left: 0


Although I will not call it here due to the high cost, I also collected the data from adding a third ghost. 

The graphs for Board 2: ![this](board2Graphs.png "Board 2 Graphs")

When adding more dots, we see that Q Table decreases significantly in effectiveness. In fact, there are only 2 times that Q Table completes the board, and one of those times, Pacman still loses a life. IDS does better than Q Table on this board, getting a higher average score, eating more dots, losing less lives. When we add 3 ghosts, each with the highest intelligence, neither IDS or Q Table is able to win


** Board 3**

Again, we will add and remove some dots and see how our algorithms do in winning the game. Here is our third board: ![this](board3.png "Board 3")

For Board 3, the Jupyter notebook takes too long to run the tests. I am able to run them on an IDE, with the results below:

[1, 1] IDS 30-average results:            Turns: 48        Score: 74        Lives Left: 3        Dots left: 0

[1, 1] Q Table results:                   Turns: 45        Score: 0         Lives Left: 0        Dots left: 4

[1, 2] IDS 30-average results:            Turns: 40        Score: 81        Lives Left: 3        Dots left: 0

[1, 2] Q Table results:                   Turns: 27        Score: 0         Lives Left: 0        Dots left: 7

[1, 3] IDS 30-average results:            Turns: 47        Score: 74        Lives Left: 3        Dots left: 0

[1, 3] Q Table results:                   Turns: 38        Score: 0         Lives Left: 0        Dots left: 2

[2, 1] IDS 30-average results:            Turns: 43        Score: 55        Lives Left: 2        Dots left: 1

[2, 1] Q Table results:                   Turns: 24        Score: 0         Lives Left: 0        Dots left: 8

[2, 2] IDS 30-average results:            Turns: 69        Score: 38        Lives Left: 1        Dots left: 3

[2, 2] Q Table results:                   Turns: 31        Score: 0         Lives Left: 0        Dots left: 2

[2, 3] IDS 30-average results:            Turns: 46        Score: 75        Lives Left: 3        Dots left: 0

[2, 3] Q Table results:                   Turns: 20        Score: 0         Lives Left: 0        Dots left: 10


And the visuals: ![this](board3Graphs.png "Board 3 Graphs")

As we saw in the last board, IDS outperforms Q Table with this board as well. Q Table is not able to fully complete any game, although it does get close when there is only one ghost. IDS is able to get a relatively high score while not losing all it's lives

# References

* [Public GitHub Repository Used for Development](https://github.com/StaticNomad/PacMan)
* [Lecture 05 - Iterative Deepening and Other Uninformed Search Methods](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/05%20Iterative%20Deepening%20and%20Other%20Uninformed%20Search%20Methods.ipynb)
* [Lecture 06 - Python Implementation of Iterative Deepening](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/06%20Python%20Implementation%20of%20Iterative%20Deepening.ipynb)
* [Lecture 15 - Reinforcement Learning for Two-Player Games](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/15%20Reinforcement%20Learning%20for%20Two-Player%20Games.ipynb) 
* [Assignemnt 02 - Iterative-Deepending Search](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/A2%20Iterative-Deepening%20Search.ipynb)
* [Assignemnt 05 - A5 Reinforcement Learning Solution to the Towers of Hanoi Puzzle](http://nbviewer.jupyter.org/url/www.cs.colostate.edu/~anderson/cs440/notebooks/A5%20Reinforcement%20Learning%20Solution%20to%20Towers%20of%20Hanoi.ipynb)